# RAG (Retrieval-Augmented Generation) Examples with DSPy

This notebook demonstrates different RAG patterns implemented using DSPy:
1. Basic RAG with single-hop retrieval
2. Multi-hop RAG for complex queries
3. Self-correcting RAG with verification
4. Progressive RAG with query refinement

In [ ]:
import sys
sys.path.append('../')

from src.config import setup_dspy, setup_retriever
from src.basic_qa import RAGBasedQA
from src.advanced_rag import MultiHopRetriever, SelfCorrectingRAG, ProgressiveRAG

# Setup DSPy and retriever
lm = setup_dspy()
retriever = setup_retriever()

## 1. Basic RAG Example
Let's start with a simple RAG-based QA system

In [ ]:
basic_rag = RAGBasedQA(retriever)

question = "What are the main components of a RAG system?"
answer = basic_rag(question)
print(f"Question: {question}\nAnswer: {answer}")

## 2. Multi-hop RAG
Now let's try a more complex query that requires multiple retrieval steps

In [ ]:
multihop = MultiHopRetriever()

complex_question = "What are the environmental impacts of electric cars, particularly regarding battery production and disposal?"
result = multihop(complex_question)

print(f"Question: {complex_question}")
print(f"Answer: {result['answer']}")
print(f"Number of contexts used: {len(result['contexts'])}")

## 3. Self-correcting RAG
This example shows how RAG can verify and correct its own outputs

In [ ]:
correcting_rag = SelfCorrectingRAG()

question = "Explain the theory of relativity and its key principles"
result = correcting_rag(question)

print(f"Question: {question}")
print(f"Answer: {result['answer']}")
print(f"Confidence: {result['confidence']}")
print(f"Attempts needed: {result['attempts']}")
print(f"Reasoning: {result['reasoning']}")

## 4. Progressive RAG
This example demonstrates how to progressively refine queries for better results

In [ ]:
progressive_rag = ProgressiveRAG()

question = "What are the implications of quantum computing on current encryption methods?"
result = progressive_rag(question)

print(f"Question: {question}")
print(f"Final Answer: {result['answer']}")
print("\nQuery Evolution:")
for i, query in enumerate(result['queries_used']):
    print(f"Step {i+1}: {query}")
print(f"\nTotal contexts used: {result['context_count']}")

# DSPy RAG (Retrieval Augmented Generation) Examples

This notebook demonstrates various RAG implementations using DSPy:
1. Basic RAG with single passage retrieval
2. Multi-hop RAG reasoning
3. Self-correcting RAG with fact verification

In [ ]:
import sys
sys.path.append('../')

import dspy
from src.config import setup_dspy, setup_retriever

# Setup DSPy and retriever
lm = setup_dspy()
retriever = setup_retriever()

## Multi-hop RAG Example
This example shows how to chain multiple retrievals for complex queries

In [ ]:
class MultiHopRAG(dspy.Module):
    def __init__(self):
        super().__init__()
        self.generate_subquestions = dspy.ChainOfThought("question -> subquestions")
        self.retrieve = dspy.Retrieve(k=2)
        self.generate_answer = dspy.ChainOfThought("context, question -> answer")
    
    def forward(self, question):
        # Break down into sub-questions
        subq = self.generate_subquestions(question=question)
        
        # Retrieve context for each sub-question
        contexts = []
        for q in subq.subquestions.split(';'):
            passages = self.retrieve(q).passages
            contexts.extend(passages)
        
        # Generate final answer
        response = self.generate_answer(context=contexts, question=question)
        return response.answer

# Create and test multi-hop RAG
multihop_rag = MultiHopRAG()
question = "What are the environmental impacts of electric vehicles, considering both manufacturing and usage?"
answer = multihop_rag(question)
print(f"Q: {question}\nA: {answer}")

## Self-correcting RAG
This example implements a RAG system that verifies its own outputs

In [ ]:
class FactCheckingRAG(dspy.Module):
    def __init__(self):
        super().__init__()
        self.retrieve = dspy.Retrieve(k=3)
        self.generate = dspy.ChainOfThought("context, question -> answer, reasoning")
        self.verify = dspy.ChainOfThought("context, answer, reasoning -> is_verified, corrections")
    
    def forward(self, question):
        # Initial retrieval and answer generation
        passages = self.retrieve(question).passages
        initial_response = self.generate(context=passages, question=question)
        
        # Verify and correct if needed
        verification = self.verify(
            context=passages,
            answer=initial_response.answer,
            reasoning=initial_response.reasoning
        )
        
        return {
            'final_answer': verification.corrections if not verification.is_verified 
                          else initial_response.answer,
            'verified': verification.is_verified
        }

# Test the fact-checking RAG
fact_checker = FactCheckingRAG()
question = "What are the main causes of climate change?"
result = fact_checker(question)
print(f"Q: {question}")
print(f"A: {result['final_answer']}")
print(f"Verified: {result['verified']}")